In [1]:
import numpy as np
import scipy.stats as si
import numpy as np
import scipy.stats as si
import scipy
import pandas as pd
from pandas import DataFrame,Series
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels
from scipy.stats import norm
from scipy import log,exp,sqrt,stats
from scipy.interpolate import interp1d
## Draw graphs in the notebook
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Black-Sholes formula:

$$Call = Se^{-qT}N(d_1) - Ke^{-rT}N(d_2)$$

where

$$d_1 = \frac{ln(\frac{S}{K}) + (r-q+\frac{1}{2}\sigma^2)T}{\sigma\sqrt{T}}$$

$$d_2 = d_1 - \sigma\sqrt{T}$$

In [0]:
def bs_call(S,K,T,sigma, r =0.1, q = 0.05):
    '''
    Preparation
    BS-formula for call options
    
    '''
    
    #S: spot price
    #K: strike price
    #T: time to maturity
    #sigma: volatility of underlying asset
    #r: interest rate
    #q: continuous dividend rate
    
    
    
    d1 = (np.log(S/K) + ( r - q + 0.5 * sigma ** 2 ) * T )/ (sigma*np.sqrt(T)) ;
    
    d2 = d1 - sigma*np.sqrt(T);
    
    return S*np.exp(-q*T)*stats.norm.cdf(d1)-K*np.exp(-r*T)*stats.norm.cdf(d2)

In [4]:
bs_call(S = 100, K = 80, T = 1, sigma = 0.30, r = 0.05, q = 0.01)

25.614621075647065

To calculate implied volatility, we need to calculate the Greek 'Vega' first.

By definition $Vega(C) = \frac{\partial{C}}{\partial{\sigma}}$
$$vega(C) = \frac{1}{\sqrt{2\pi}}Se^{-qT}e^{\frac{-d_1^2}{2}}\sqrt{T}$$

In [0]:
def vega_calculation(S,K,T,sigma, r=0.1, q=0.05):
    '''
    calculate vega for dividend-paying asset
    S: spot price
    K: strike price
    T: time to maturity
    sigma: volatility of underlying asset
    r: interest rate
    q: continuous dividend rate
    
    '''
    
    d1 = (np.log(S/K)+( r - q + 0.5 * sigma ** 2 ) * T )/(sigma*np.sqrt(T));
    
    vega = 1 / np.sqrt(2 * np.pi) * S * np.exp(-q * T) * np.exp(-(d1 ** 2) * 0.5) * np.sqrt(T)
    
    return vega

In [0]:
def newton_vol_call(S, K, T, sigma,C, r, q):
    '''
    calculate implied volatiliy via newton method
    S: spot price
    K: strike price
    T: time to maturity
    sigma: volatility of underlying asset
    C: Call value
    r: interest rate
    q: continuous dividend rate
    
    '''
    
    d1 = (np.log(S/K)+( r - q + 0.5 * sigma ** 2 ) * T )/(sigma*np.sqrt(T));
    
    d2 = d1-sigma*np.sqrt(T);
    

    
    vega = vega_calculation(S,K,T,sigma,r,q);
   
    epsilon = 0.000001
    
    x0 = sigma
    xnew  = x0
    xold = x0 - 1
        
    while abs(xnew - xold) > epsilon:
    
        xold = xnew
        xnew = xnew - (bs_call(S,K,T,xold,r,q) - C) / vega_calculation(S,K,T,xold,r,q)
        
        implied_vol = abs(xnew)
        return implied_vol

In [0]:
testcall = bs_call(120, 100, 1, 0.3, 0.1, 0.05)
testcall

27.471915694885013

In [0]:
newton_vol_call(120, 100, 1,0.9, testcall, 0.1, 0.05)    

0.3078078676402214